In [2]:
import numpy as np
import torch
from env import MultiAgentEnv
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
import pandas as pd
import json
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque

In [ ]:
X_train_df = pd.read_csv('/Users/louisedurand-janin/Documents/GitHub/HrFlow_Data_Challenge/data/X_train.csv')
X_train_df['employee embedding'] = X_train_df['employee embedding'].apply(lambda x: np.array(json.loads(x), dtype=np.float32))
X_train_df['company embedding'] = X_train_df['company embedding'].apply(lambda x: np.array(json.loads(x), dtype=np.float32))

In [ ]:
y_train = np.load('data/y_train.npy', allow_pickle=True)

In [ ]:
# Convertir les colonnes d'embeddings en tensors PyTorch
employee_embedding_tensor = torch.tensor(np.vstack(X_train_df['employee embedding'].values), dtype=torch.float32)
company_embedding_tensor = torch.tensor(np.vstack(X_train_df['company embedding'].values), dtype=torch.float32)

# Concaténer les deux tensors le long de la dimension appropriée (axis=1 pour ajouter des colonnes)
combined_tensor = torch.cat([employee_embedding_tensor, company_embedding_tensor], dim=1)

In [ ]:


class DQNAgent(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNAgent, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0   # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001

        # Neural Net for Deep-Q learning Model
        self.fc1 = nn.Linear(state_size, 24)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(24, 24)
        self.fc3 = nn.Linear(24, action_size)
        
        self.optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)

    def forward(self, state):
        x = self.relu(self.fc1(state))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.FloatTensor(state).unsqueeze(0)
        act_values = self.forward(state)
        return np.argmax(act_values.cpu().data.numpy())

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            state = torch.FloatTensor(state).unsqueeze(0)
            next_state = torch.FloatTensor(next_state).unsqueeze(0)
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.forward(next_state).cpu().data.numpy())
            target_f = self.forward(state)
            target_f[0][action] = target
            self.optimizer.zero_grad()
            loss = nn.MSELoss()(target_f, target_f)  # Note: You might want to adjust this loss calculation.
            loss.backward()
            self.optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
